In [ ]:
from transformers import LlavaForConditionalGeneration, AutoProcessor
from PIL import Image
import torch

# Model & processor
model_path = "chaoyinshe/llava-med-v1.5-mistral-7b-hf"

model = LlavaForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_path)

def explainXray(classification_result, image_path):
  image = Image.open(image_path).convert("RGB")

  # Build prompt
  question_text = f"This chest X-ray suggests ${classification_result} for pneumonia. What is the main radiographic finding in the image that supports this diagnosis?"
  prompt = f"USER: <image>\n{question_text} ASSISTANT:"

  # Prepare input
  inputs = processor(
      images=[image],
      text=prompt,
      return_tensors="pt"
  ).to(model.device, torch.float16)

  # Generate output
  with torch.inference_mode():
      out = model.generate(
          **inputs,
          max_new_tokens=256,
          do_sample=False,
          temperature=0.0,
          pad_token_id=processor.tokenizer.eos_token_id
      )

  # Decode cleanly
  response = processor.decode(out[0], skip_special_tokens=True)
  print("\n--- Chest X-ray Findings ---\n")
  print(response.strip())

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/686 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [ ]:
explainXray('positive', 'positive/xray1.png')

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- Chest X-ray Findings ---

USER: 
This chest X-ray suggests $positive for pneumonia. What is the main radiographic finding in the image that supports this diagnosis? ASSISTANT: The main radiographic finding in the image that supports the diagnosis of pneumonia is the presence of bilateral infiltrates. These infiltrates are areas of increased density in the lung tissue, which can be indicative of inflammation or infection.


In [ ]:
explainXray('positive', 'positive/xray2.png')


--- Chest X-ray Findings ---

USER: 
This chest X-ray suggests $positive for pneumonia. What is the main radiographic finding in the image that supports this diagnosis? ASSISTANT: The main radiographic finding in the image that supports the diagnosis of pneumonia is the presence of bilateral infiltrates, which are areas of increased density in the lung tissue. These infiltrates can be seen in both lungs and are often indicative of inflammation or infection, such as pneumonia.


In [ ]:
explainXray('negative', 'negative/xray1.png')


--- Chest X-ray Findings ---

USER: 
This chest X-ray suggests $negative for pneumonia. What is the main radiographic finding in the image that supports this diagnosis? ASSISTANT: The main radiographic finding in the image that supports the diagnosis of no pneumonia is the absence of any significant abnormalities in the lung fields. This means that the lungs appear to be clear and free of any signs of infection or inflammation.


In [ ]:
explainXray('negative', 'negative/xray2.png')


--- Chest X-ray Findings ---

USER: 
This chest X-ray suggests $negative for pneumonia. What is the main radiographic finding in the image that supports this diagnosis? ASSISTANT: The main radiographic finding in the image that supports the diagnosis of $negative for pneumonia is the presence of a right-sided pleural effusion. This finding is consistent with the patient's clinical presentation and the absence of pneumonia.
